In [1]:
import pandas as pd
from model_monitoring import ModelMonitoring

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\User\AppData\Local\Programs\Python\Python311\

In [2]:
incoming_df = pd.read_csv("../data/incoming_data_cleaned.csv")
test_df = pd.read_csv("../data/test_data_cleaned.csv")
incoming_features_df_processed = pd.read_csv("../data/incoming_features_df_processed.csv")
test_features_df_processed = pd.read_csv("../data/test_features_df_processed.csv")

In [3]:
model_monitoring = ModelMonitoring()

In [4]:
model_monitoring.data_check(test_df, incoming_df, test_features_df_processed, incoming_features_df_processed)

In [5]:
model_monitoring.write_to_config()

After running write_to_config, proceed to config.json file. It will show a dictionary with all the column names as keys and their respective thresholds as values. User can change the thresholds in this config file for the individual columns to suit their requirements for the respective columns

In [6]:
# model_monitoring.feature_drift_report(test_df.drop(columns=['target', 'prediction']), incoming_df.drop(columns=['target', 'prediction']), 'json')
model_monitoring.feature_drift_report(test_df.drop(columns=['target', 'prediction']), incoming_df.drop(columns=['target', 'prediction']), 'html')

In [7]:
"""
For categorical: 
chisquare, z, fisher_exact, g_test, TVD
For numerical: 
ks, wasserstein, anderson, cramer_von_mises, mannw, ed, es, t_test, emperical_mmd
For both categorical and numerical: 
kl_div, psi, jensenshannon, hellinger
for more information on the stats test, please refer to: 
https://docs.evidentlyai.com/user-guide/customization/options-for-statistical-tests
"""
# model_monitoring.prediction_drift_report(test_df, incoming_df, 'json')
model_monitoring.prediction_drift_report(test_df, incoming_df, 'chisquare', 0.05, 'html')